In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Apply Augmentation to Oversample the minority class

import os
import cv2
import numpy as np

def choose_random_augmentation():
    return np.random.choice(['rotate', 'flip', 'filter', 'none'])

def apply_augmentation(frame, augmentation_type):
    if augmentation_type == 'rotate':
        rows, cols, _ = frame.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), 45, 1) #implements rotation bij 45 degrees
        augmented_frame = cv2.warpAffine(frame, rotation_matrix, (cols, rows))
    elif augmentation_type == 'flip':
        # Use cv2.flip with a fixed flip code to ensure consistency
        augmented_frame = cv2.flip(frame, 1)  # 1 corresponds to horizontal flip
    elif augmentation_type == 'filter':
        augmented_frame = cv2.GaussianBlur(frame, (5, 5), 0)
    else:
        augmented_frame = frame  # 'none' augmentation

    return augmented_frame

def apply_augmentation_to_video(video_frames, augmentation_type):
    augmented_video_frames = []
    for frame in video_frames:
        augmented_frame = apply_augmentation(frame, augmentation_type)
        augmented_video_frames.append(augmented_frame)
    return augmented_video_frames

def save_augmented_video(input_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(input_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Read all frames of the video
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    # Check if filename contains 'a9_100' to determine augmentation
    if 'a9_100' in input_path:
        augmentation_type = choose_random_augmentation()
        augmented_frames = apply_augmentation_to_video(frames, augmentation_type)

        output_name = f"ROS{os.path.splitext(os.path.basename(input_path))[0]}2.mp4"
        output_path = os.path.join(output_folder, output_name)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        for frame in augmented_frames:
            out.write(frame)

        cap.release()
        out.release()

# Specify input and output folders
input_folder_path = '/content/drive/MyDrive/Colab Notebooks /frames/train_full'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks /frames/train_full_ROS2'

# Load and augment videos
for video_file in os.listdir(input_folder_path):
    if video_file.endswith(".mp4"):
        input_path = os.path.join(input_folder_path, video_file)
        save_augmented_video(input_path, output_folder_path)


In [ ]:
#copy the contents of the original training set to the oversampled folder
import shutil
import os

def copy_folder_contents(input_folder_path, output_folder_path):
    try:
        # Create the destination folder if it doesn't exist
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        # Copy the contents of the source folder to the destination folder
        for item in os.listdir(source_folder):
            source_item = os.path.join(source_folder, item)
            destination_item = os.path.join(destination_folder, item)

            if os.path.isdir(source_item):
                shutil.copytree(source_item, destination_item, symlinks=False, ignore=None)
            else:
                shutil.copy2(source_item, destination_item)

        print(f"Contents of '{source_folder}' successfully copied to '{destination_folder}'")

    except Exception as e:
        print(f"Error: {e}")

# Example usage
# Specify input and output folders  #change to desired folder path for all the frames
source_folder = '/content/drive/MyDrive/Colab Notebooks /frames/train_full'
destination_folder = '/content/drive/MyDrive/Colab Notebooks /frames/train_full_ROS2'

copy_folder_contents(source_folder, destination_folder)



Contents of '/content/drive/MyDrive/Colab Notebooks /frames/train_full' successfully copied to '/content/drive/MyDrive/Colab Notebooks /frames/train_full_ROS2'


In [ ]:
#check if the videos of class 9 have more values

import os

def count_videos_with_a9_100(folder_path):
    video_count = 0

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.mp4') and 'a9_100' in file_name:
            video_count += 1

    return video_count

# Example usage:
folder_path = '/content/drive/MyDrive/Colab Notebooks /frames/train_full'  # Replace this with your folder path
count = count_videos_with_a9_100(folder_path)
print(f"Number of videos with 'a9_100' in train_full: {count}")


folder_path = '/content/drive/MyDrive/Colab Notebooks /frames/train_full_ROS2'
count = count_videos_with_a9_100(folder_path)
print(f"Number of videos with 'a9_100' in train_full_ROS2: {count}")

Number of videos with 'a9_100' in train_full: 17
Number of videos with 'a9_100' in train_full_ROS2: 34
